In [ ]:
import os
import requests
import base64
import json
from flask import Flask

# Be explicit with the auth imports to avoid attribute errors
import google.auth
from google.auth.transport import requests as google_auth_requests

# Initialize the Flask app
app = Flask(__name__)

# --- CONFIGURATION ---
PROJECT_ID = os.environ.get('GCP_PROJECT', 'robotic-fuze-463700-n5')
TOPIC_ID = os.environ.get('PUBSUB_TOPIC', 'nightly-data-pull-topic')

@app.route("/", methods=["POST"])
def trigger_pipeline():
    """
    Receives a POST request (from Cloud Scheduler) and publishes a
    message to a Pub/Sub topic to start the data processing pipeline.
    """
    try:
        # Get the default credentials from the environment
        credentials, project = google.auth.default(
            scopes=["https://www.googleapis.com/auth/pubsub"]
        )

        # Create a transport-specific request object using our explicit import
        auth_req = google_auth_requests.Request()
        credentials.refresh(auth_req)
        token = credentials.token

        # Construct the URL for the Pub/Sub REST API
        url = f"https://pubsub.googleapis.com/v1/projects/{PROJECT_ID}/topics/{TOPIC_ID}:publish"

        # The message can be simple, its presence is the trigger.
        message_data = b'Start data pipeline'
        base64_encoded_message = base64.b64encode(message_data).decode('utf-8')

        # Construct the request body
        payload = { "messages": [ { "data": base64_encoded_message } ] }

        # Set the authorization header
        headers = {
            "Authorization": f"Bearer {token}",
            "Content-Type": "application/json; charset=utf-8"
        }

        # Make the POST request to the API
        response = requests.post(url, data=json.dumps(payload), headers=headers)
        response.raise_for_status()

        message_ids = response.json().get('messageIds', [])
        print(f"Published message(s) {message_ids} to {TOPIC_ID}.")

        return f"Successfully triggered pipeline (Message IDs: {message_ids}).", 202

    except Exception as e:
        print(f"Error publishing to Pub/Sub via REST API: {e}")
        return "Error triggering pipeline.", 500

if __name__ == "__main__":
    # Get the port number from the environment variable PORT
    port = int(os.environ.get("PORT", 8080))
    # Run the app, listening on all available network interfaces
    app.run(debug=True, host="0.0.0.0", port=port)
